# Pure Python Pipeline
Ripple noise removal, motion correction, trace deconvoloution and extraction in one notebook. This should replace demo_pipeline as the standard analysis notebook! 

## TODO list (from demo_pipeline)
- TODO: hdf5_to_numpy as reverse of numpy_to_hdf5 (RNR export step)
- TODO: check demo_motion_correction.ipynb for evaluation of motion correction
- TODO: appropriate file variable names (fname_new ... are NOT appropriate).
- TODO: save file names should be more meaningful, for easier cleanup.
- TODO: rnr save file: should have rnr in the hdf5 file name, otherwise too confusing!
- TODO: create a small txt file with the file names and their purpose (whether they can be deleted, how to use them, what do they contain) at some point (early) in the analysis, save it in export folder.
- TODO: cnm.estimates.idx_components and idx_components_bad are removed at one step. Instead, there is cnm.estimates.discarded_components. Do we need manual accept/reject?
- FIXME: _pars.json and _results.hdf5 contain nd2 file name twice: T301_tmev_d1T301_tmev_d1.270820.1110_22-10-20_14-18-40_pars.json and T301_tmev_d1T301_tmev_d1.270820.1110_22-10-20_14-18-40_results.hdf5
- IMPORTANT: make it more convenient to enter pipeline from any point. This includes defining parameters in one location, naming variables appropriately (F memmap, C memmap, nd2 file, hd5 file...) so user is aware which file they are supposed to open at which point of entry into analysis!
- memmap_ results in conflicting names if recordings are from same day. Include date time of analysis in name? Not a big problem as it is temporary file
- Study parallel processing of caiman (start server step, cleaning up server). It might be useful for RNR too.
- Evaluating components: cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview), the contents of model/ in CaImAn are used but looking for the model files in another directory (in my case, Users/Bence/caiman_data/model/)
- Export h5 file should have date time in filename to avoid overwriting. Both raw data and final results!
- Plot with slider: watch all the frames, compare RNR and original, then MC and original/RNR... QC
- Save RNR directly to memmap (opening as caiman movie, save to memmap?)? Although the problem is how slow RNR is...
- Maybe working with numpy array in motion correction (movie.motion_correct) is not that bad? Although no parameters...
- Plot frame before RNR and after RNR to set parameters... Interactive?
- Read Tips on analysis: https://caiman.readthedocs.io/en/master/CaImAn_Tips.html#motion-correction-tips
- RNR results in 4x size (uint16 to float64)! Need to clean up or use uint16 again.
- Check 2-channel recordings. Might want to save red channel, too, for matching?
- Save memmap files is inconsistent in naming (C order is memmap__d1_512_d2_512_d3_1_order_C_frames_577_ instead of T386_20211202_green_ex_els__d1_512_d2_512_d3_1_order_C_frames_577_)
- Include nd2 to h5 here (from nd2 to multipage tiff test.ipynb)
- It takes a lot of time to open nd2 file. Useful to copy data to be analyzed to local HDD on a previous day?
- way to manually reject/accept components
- IMPORTANT: https://caiman.readthedocs.io/en/master/On_file_types_and_sizes.html caiman works best when files are 1-2 GB big! It means we might want to split them in small pieces, or make sure they are multi-page tiff files!

## Import packages

In [1]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [2]:
from RippleNoiseRemoval import RNR
import labrotation.file_handling as fh
import h5py
from time import time

import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
from labrotation import file_handling as fh
from copy import deepcopy
try:
    cv2.setNumThreads(0)
except():
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour

from movie_splitting import numpy_to_hdf5
bpl.output_notebook()

Loading BokehJS ...

In [3]:
log_fname = fh.choose_dir_for_saving_file("Select folder to save log file", fh.get_filename_with_date("caim_log", ".txt"))
print(f"Saving log file to\n{log_fname}")

Saving log file to
D:\Presentation\mocofail\caim_log_22-12-06_00-07-19.txt



## Set up logging (optional)

In [4]:
logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",\
                    filename=log_fname,
                    level=logging.WARNING)

## Set input and output files

In [5]:
nd2_fpath = fh.open_file("Select nd2 file") #"D:/PhD/Data/T386_MatlabTest/T386_20211202_green.nd2"
print(f"Input file selected: {nd2_fpath}")

Input file selected: Y:\AG-Wenzel\Group\tmev\T333\T333_tmev_d2\T333_tmev_d2_21102020_FOV_005.nd2


In [6]:
# save in same folder as nd2 file
export_folder = fh.open_dir("Select folder to save results", True)
# export_fname: get rid of .nd2 extension, append date and .h5 extension
export_fname = fh.get_filename_with_date(os.path.splitext(os.path.split(nd2_fpath)[1])[0] + "_rnr_", ".hdf5")
export_hd5_fpath = os.path.join(export_folder, export_fname) # nd2_fpath.split("/")[-1][:-4] + "_exp.h5"
print(f"Export file selected: {export_hd5_fpath}")

Export file selected: D:\Presentation\mocofail\T333_tmev_d2_21102020_FOV_005_rnr__22-12-06_00-07-50.hdf5


## Ripple Noise Removal

In [7]:
win = 40
amplitude_threshold = 10.8

In [8]:
rnr = RNR(win, amplitude_threshold) 

In [9]:
begin_end_frames = [10287,10972]

In [10]:
#begin_end_frames = None  # (begin, end): if want to work with part of the file
t0_open = time()
rnr.open_recording(nd2_fpath, begin_end_frames)  # opens usual recording size (8.8-9 GB) in about 830 s
print(f"File opened in {time() - t0_open} s")

C:\Users\Bence\anaconda3\envs\2p-py\lib\site-packages\pims\base_frames.py:472: UserWarning: Please call FramesSequenceND.__init__() at the start of thethe reader initialization.
  warn("Please call FramesSequenceND.__init__() at the start of the"


Opened recording 512x512, 686 frames. Initialized empty results array.
File opened in 96.94191336631775 s


In [11]:
t0_single = time()
rnr_data = rnr.rnr_singlethread()  # a bit faster than opening file, around 500s for 8.8-9 GB
t1_single = time()
print(f"RNR single thread finished in {t1_single - t0_single} s")
print(f"Result is a {type(rnr_data)} with datatype {rnr_data.dtype}")
print(f"Shape: {rnr_data.shape[0]} frames of {rnr_data.shape[1]}x{rnr_data.shape[2]} pixels")

D:\Codebase\2p-py\RippleNoiseRemoval.py:45: RuntimeWarning: divide by zero encountered in log
  ampl_image = np.log(np.abs(freq_image))


RNR completed.
RNR single thread finished in 18.964435577392578 s
Result is a <class 'numpy.ndarray'> with datatype float64
Shape: 686 frames of 512x512 pixels


### Export RNR movie to hd5 file.
The reason to this otherwise unnecessary step is that motion correction cannot work from numpy array... Or at least the movie.motion_correct() does not have many options. See https://caiman.readthedocs.io/en/master/core_functions.html#movie-handling motion_correct

In [12]:
numpy_to_hdf5(rnr_data, export_hd5_fpath)
#dataset_name = "mov"  # var_name_hdf5 in various functions refers to this name! Default is always mov.
#with h5py.File(export_hd5_fpath, 'w') as hf:
#    dataset = hf.create_dataset(dataset_name, shape=rnr_data.shape, dtype=np.float64)  # TODO: float64 is much larger file!
#    for i_frame in range(rnr_data.shape[0]):
#        dataset[i_frame] = rnr_data[i_frame]


numpy_to_hdf5: Single output filename detected.
numpy_to_hdf5: No splitting will be performed.
Creating 1 file(s):
	D:\Presentation\mocofail\T333_tmev_d2_21102020_FOV_005_rnr__22-12-06_00-07-50.hdf5
Done.


['D:\\Presentation\\mocofail\\T333_tmev_d2_21102020_FOV_005_rnr__22-12-06_00-07-50.hdf5']

## Motion Correction

### Optional: split up video into segments to process and skip
Important: the numbering should correspond to the nd2 indexing, i.e. from 1 to n_frames! No zero-indexing here!

In [13]:
#frames_begin_end = [(1,109),(110,135),(136,448),(449,499), (500, 577)]
#flag_moco = [True, False, True, False, True]
# t333 tmev d2 fiv 005
#frames_begin_end = [(1, 9862), (9863,9908), (9909, 10085), (10086, 10115), (10115, 10241), (10242, 10276), (10277, 10290), (10291, 10551), (10552, 10620), (10621, 11748), (11749, 17628)]
#flag_moco = [True, False, True, False, True, False, True, False, True, False, True]
# TODO: check here for consistency of these variables!
# (9800, 12200)
#frames_begin_end = [(1, 491), (492, 752), (753, 821), (822, 1949), (1950, 2400)]
#flag_moco = [True, False, True, False, True]
frames_begin_end = [(1, )]
flag_moco = [True]

### Set output file

In [14]:
if not("export_hd5_fpath" in locals()):
    export_hd5_fpath = fh.open_file("Choose hd5 file to open")
if export_hd5_fpath.split(".")[-1] != "hdf5":
    print(f"Invalid hd5 file:\n{export_hd5_fpath}\nChoose a valid hd5 file!")
    export_hd5_fpath = fh.open_file("Choose hd5 file to open")
fnames = [export_hd5_fpath]
print(f"Going to perform MoCo on {fnames}")
assert export_hd5_fpath.split(".")[-1] == "hdf5", f"Invalid file extension: .{export_hd5_fpath.split('.')[-1]}, expected .hdf5"

Going to perform MoCo on ['D:\\Presentation\\mocofail\\T333_tmev_d2_21102020_FOV_005_rnr__22-12-06_00-07-50.hdf5']


### Optional: Play the movie

In [15]:
display_movie = False
if display_movie:
    ds_ratio = 0.2
    movie.resize(1, 1, ds_ratio).play(
        q_max=99.5, fr=30, magnification=2)  # this should not change size of movie itself

### Setup some parameters
We set some parameters that are relevant to the file, and then parameters for motion correction, processing with CNMF and component quality evaluation. Note that the dataset `Sue_2x_3000_40_-46.tif` has been spatially downsampled by a factor of 2 and has a lower than usual spatial resolution (2um/pixel). As a result several parameters (`gSig, strides, max_shifts, rf, stride_cnmf`) have lower values (halved compared to a dataset with spatial resolution 1um/pixel).

In [16]:
# dataset dependent parameters
fr = 15                             # imaging rate in frames per second
decay_time = 0.4                    # length of a typical transient in seconds

# motion correction parameters
strides = (48, 48)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)         # overlap between pathes (size of patch strides+overlaps)
max_shifts = (6,6)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

# parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system
gnb = 2                     # number of global background components
merge_thr = 0.85            # merging threshold, max correlation allowed
rf = 15                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 6             # amount of overlap between the patches in pixels
K = 4                       # number of components per patch
gSig = [4, 4]               # expected half size of neurons in pixels
method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
ssub = 1                    # spatial subsampling during initialization
tsub = 1                    # temporal subsampling during intialization

# parameters for component evaluation
min_SNR = 2.0               # signal to noise ratio for accepting a component
rval_thr = 0.85              # space correlation threshold for accepting a component
cnn_thr = 0.99              # threshold for CNN based classifier
cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected

### Create a parameters object
You can creating a parameters object by passing all the parameters as a single dictionary. Parameters not defined in the dictionary will assume their default values. The resulting `params` object is a collection of subdictionaries pertaining to the dataset to be analyzed `(params.data)`, motion correction `(params.motion)`, data pre-processing `(params.preprocess)`, initialization `(params.init)`, patch processing `(params.patch)`, spatial and temporal component `(params.spatial), (params.temporal)`, quality evaluation `(params.quality)` and online processing `(params.online)`

In [17]:
if "fnames" not in locals():
    fnames = fh.open_file("No hd5 file selected. Choose corresponding hd5 file!")
opts_dict = {'fnames': fnames, 
            'fr': fr,
            'decay_time': decay_time,
            'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid,
            'p': p,
            'nb': gnb,
            'rf': rf,
            'K': K, 
            'stride': stride_cnmf,
            'method_init': method_init,
            'rolling_sum': True,
            'only_init': True,
            'ssub': ssub,
            'tsub': tsub,
            'merge_thr': merge_thr, 
            'min_SNR': min_SNR,
            'rval_thr': rval_thr,
            'use_cnn': True,
            'min_cnn_thr': cnn_thr,
            'cnn_lowest': cnn_lowest,
            'var_name_hdf5': 'data',}  # FIXME: does not work! Check where does this setting get lost?

opts = params.CNMFParams(params_dict=opts_dict)

### Setup a cluster
To enable parallel processing a (local) cluster needs to be set up. This is done with a cell below. The variable `backend` determines the type of cluster used. The default value `'local'` uses the multiprocessing package. The `ipyparallel` option is also available. More information on these choices can be found [here](https://github.com/flatironinstitute/CaImAn/blob/master/CLUSTER.md). The resulting variable `dview` expresses the cluster option. If you use `dview=dview` in the downstream analysis then parallel processing will be used. If you use `dview=None` then no parallel processing will be employed.

In [18]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [19]:
mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))

In [20]:
# ALTERNATIVE to exporting h5 and importing it again!
"""
Args:
            max_shift_w,max_shift_h: maximum pixel shifts allowed when correcting
                                     in the width and height direction

            template: if a good template for frame by frame correlation exists
                      it can be passed. If None it is automatically computed

            method: depends on what is installed 'opencv' or 'skimage'. 'skimage'
                    is an order of magnitude slower

            num_frames_template: if only a subset of the movies needs to be loaded
                                 for efficiency/speed reasons
                                 
max_shift_w=5,
max_shift_h=5,
num_frames_template=None,
template=None,
method: str = 'opencv',
remove_blanks: bool = False,
interpolation: str = 'cubic'
"""

# movie.motion_correct()   # this might change movie itself! Alternative: extract_shifts, apply_shifts

"\nArgs:\n            max_shift_w,max_shift_h: maximum pixel shifts allowed when correcting\n                                     in the width and height direction\n\n            template: if a good template for frame by frame correlation exists\n                      it can be passed. If None it is automatically computed\n\n            method: depends on what is installed 'opencv' or 'skimage'. 'skimage'\n                    is an order of magnitude slower\n\n            num_frames_template: if only a subset of the movies needs to be loaded\n                                 for efficiency/speed reasons\n                                 \nmax_shift_w=5,\nmax_shift_h=5,\nnum_frames_template=None,\ntemplate=None,\nmethod: str = 'opencv',\nremove_blanks: bool = False,\ninterpolation: str = 'cubic'\n"

### Perform motion correction and save as C-order memmap
The filename is mc.fname_tot_els

## Perform MoCo on whole movie first, do not save the results yet

In [21]:
#%%capture
#%% Run piecewise-rigid motion correction using NoRMCorre
mc.motion_correct(save_movie=False)

## Drop calculated shift values for frames where MoCo should not be performed

In [22]:
from copy import deepcopy

In [23]:
x_shifts_els = deepcopy(mc.x_shifts_els)
y_shifts_els = deepcopy(mc.y_shifts_els)

In [24]:
x_shape = x_shifts_els[0].shape
y_shape = y_shifts_els[0].shape
for i_piece, frames_tuple in enumerate(frames_begin_end):
    if not flag_moco[i_piece]:  # skip this piece = set shifts to zero
         for i_frame in range(frames_tuple[0] -1 , frames_tuple[1]):  # include last frame as well
            x_shifts_els[i_frame] = np.zeros(x_shape)
            y_shifts_els[i_frame] = np.zeros(y_shape)

In [25]:
mc.x_shifts_els = x_shifts_els
mc.y_shifts_els = y_shifts_els

In [26]:
work_folder = os.path.split(fnames[0])[0]
print(f"Changing work folder to {work_folder}, this is where moco result will be saved")

Changing work folder to D:\Presentation\mocofail, this is where moco result will be saved


In [27]:
os.chdir(work_folder)

In [28]:
# should save in C order because cm.load_memmap() takes C-memmap. However, as the opening and closing of memmap files is 
# so confusing, I decided to try to copy the original demo_pipeline jupyter notebook as closely as I can.
exp_fname = mc.apply_shifts_movie(fnames, save_memmap=True,order="F")
print(exp_fname)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.44s/it]


MC_d1_512_d2_512_d3_1_order_F_frames_686_.mmap


In [29]:
mc_mmap = os.path.join(work_folder, exp_fname)

In [30]:
m_els = cm.load(mc_mmap)
border_to_0 = 0 if mc.border_nan is 'copy' else mc.border_to_0  # FIXME: gives warning, should use "==" with literals
    # maximum shift to be used for trimming against NaNs

<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Bence\AppData\Local\Temp\ipykernel_1620\1449429027.py:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
  border_to_0 = 0 if mc.border_nan is 'copy' else mc.border_to_0  # FIXME: gives warning, should use "==" with literals


# Save tif file (Optional, for checking MoCo)

In [31]:
# TODO: float64 causes 4x file size! (35-36 GB instead of 8-9 GB)
m_els.save(os.path.join(work_folder, 'movie_mocofail2.tif'))

### Optional: show comparison with original movie

In [ ]:
#%% compare with original movie
display_movie = False  # TODO: does not seem to work. Create own function to show result?
if display_movie:
    m_orig = cm.load_movie_chain(fnames)
    ds_ratio = 0.2
    cm.concatenate([m_orig.resize(1, 1, ds_ratio) - mc.min_mov*mc.nonneg_movie,
                    m_els.resize(1, 1, ds_ratio)], 
                   axis=2).play(fr=60, gain=15, magnification=2, offset=0)  # press q to exit

## Save now as C memmap, apply border_to_0

In [ ]:
# this is a very stupid step, basically save the results again (mc_mmap) but with border_to_0 param in addition
moco_full_c_memmap = cm.save_memmap([mc_mmap], base_name='memmap', order='C',border_to_0=border_to_0, dview=dview) # exclude borders

In [ ]:
moco_full_c_memmap  # this is the split file that swe can handle just the same as fname_new in the original notebook.

# Work with MoCo-exported C memmap

In [ ]:
fname_new = moco_full_c_memmap

In [ ]:
if 'fname_new' not in locals():
    fname_new = fh.open_file("Select C-memmap file.")
print(f"Working with C-memmap\n{fname_new}")

In [ ]:
# now load the file
Yr, dims, T = cm.load_memmap(fname_new)
images = np.reshape(Yr.T, [T] + list(dims), order='F') 
    #load frames in python format (T x X x Y)

In [ ]:
resolution = images[0].shape
resolution

In [ ]:
images.shape

### Create ndarray for concatenated "moco" segments
i.e. parts of the video where moco was used. The other segments contain weird signal that hinder the moco and CNMF.

In [ ]:
total_len = 0
len_concat = 0  # length of concatenated moco-segments video
for i_segment, frames_tuple in enumerate(frames_begin_end):
    len_segment = frames_tuple[1] - frames_tuple[0] + 1
    if flag_moco[i_segment]:  # if moco was applied, add segment
        len_concat += len_segment
        
    total_len += len_segment
        
len_concat

In [ ]:
moco_split_memmap_fname = cm.paths.memmap_frames_filename("moco_split", resolution, len_concat, "C")
moco_split_memmap_fpath = os.path.join(work_folder, moco_split_memmap_fname)
print(f"Split-up moco C memmap for CNMF will be saved as\n\t{moco_split_memmap_fpath}")

In [ ]:
Yr.shape

In [ ]:
data_concatenated = np.zeros(shape=(len_concat, resolution[0], resolution[1]), dtype=images.dtype)
i_concat = 0  # index of frame in concatenated data
for i_seq, sequence in enumerate(frames_begin_end):  # loop through split parts, moco and non-moco
    if flag_moco[i_seq]:  # if moco part, append frames to concatenated data
        for i_frame in range(sequence[0] - 1, sequence[1]):  # the tuples are indices starting with 1, so subtract 1. Also both inclusive, ccf. range() which is [inclusive, exclusive)
            data_concatenated[i_concat] = images[i_frame]
            i_concat += 1

In [ ]:
#TODO: try to save C-type memmap, then use cell below to open, then compare images2 and images. Should have same dimensions (except T 500 instead of 577)
moco_split_c_memmap = cm.save_memmap([data_concatenated], base_name='mmsplit', order='C',border_to_0=border_to_0, dview=dview) # exclude borders

In [ ]:
moco_split_c_memmap

In [ ]:
# now load the file
Yr2, dims2, T2 = cm.load_memmap(moco_split_c_memmap)
images2 = np.reshape(Yr2.T, [T2] + list(dims2), order='F') 
    #load frames in python format (T x X x Y)

In [ ]:
images.shape

In [ ]:
images2.shape

In [ ]:
# need to save data with non-moco cut out as separate mmap file:
# if does not work, from caiman.mmapping import save_memmap
#FIXME: this does not work, saves 2D data
#concat_fname = cm.save_memmap(filenames = [data_concatenated], base_name="concat", order="C")

### Clean up memory now

In [ ]:
#%% restart cluster to clean up memory
if "dview" in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
cnmf_images = images  # images2 if split data is used

### Run CNMF on patches in parallel

In [ ]:
%%capture
#%% RUN CNMF ON PATCHES
# First extract spatial and temporal components on patches and combine them
# for this step deconvolution is turned off (p=0). If you want to have
# deconvolution within each patch change params.patch['p_patch'] to a
# nonzero value
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview, nb_patch = 4)
cnm = cnm.fit(cnmf_images)

In [ ]:
cnmf_images.shape

In [ ]:
#%% plot contours of found components
Cn = cm.local_correlations(cnmf_images.transpose(1,2,0))
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours_nb(img=Cn)

### Inspecting the results
Briefly inspect the results by plotting contours of identified components against correlation image.
The results of the algorithm are stored in the object `cnm.estimates`. More information can be found in the definition of the `estimates` object and in the [wiki](https://github.com/flatironinstitute/CaImAn/wiki/Interpreting-Results).

## Re-run (seeded) CNMF  on the full Field of View  
You can re-run the CNMF algorithm seeded on just the selected components from the previous step. Be careful, because components rejected on the previous step will not be recovered here.

In [ ]:
%%capture
#%% RE-RUN seeded CNMF on accepted patches to refine and perform deconvolution 
cnm2 = cnm.refit(cnmf_images, dview=dview)

## Component Evaluation

The processing in patches creates several spurious components. These are filtered out by evaluating each component using three different criteria:

- the shape of each component must be correlated with the data at the corresponding location within the FOV
- a minimum peak SNR is required over the length of a transient
- each shape passes a CNN based classifier

In [ ]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

# if performed re-run:
if "cnm2" in locals():
    cnm2.estimates.evaluate_components(cnmf_images, cnm2.params, dview=dview)
else:
    cnm.estimates.evaluate_components(cnmf_images, cnm.params, dview=dview)

Plot contours of selected and rejected components

In [ ]:
#%% PLOT COMPONENTS
if "cnm2" in locals():
    cnm2.estimates.plot_contours_nb(img=Cn, idx=cnm2.estimates.idx_components)
else:
    cnm.estimates.plot_contours_nb(img=Cn, idx=cnm.estimates.idx_components)

View traces of accepted and rejected components. Note that if you get data rate error you can start Jupyter notebooks using:
'jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10'

In [ ]:
# accepted components
if "cnm2" in locals():
    cnm2.estimates.nb_view_components(img=Cn, idx=cnm2.estimates.idx_components)
else:
    cnm.estimates.nb_view_components(img=Cn, idx=cnm.estimates.idx_components)

In [ ]:
# rejected components
if "cnm2" in locals():
    if len(cnm2.estimates.idx_components_bad) > 0:
        cnm2.estimates.nb_view_components(img=Cn, idx=cnm2.estimates.idx_components_bad)
    else:
        print("No components were rejected.")
else:
    if len(cnm.estimates.idx_components_bad) > 0:
        cnm.estimates.nb_view_components(img=Cn, idx=cnm.estimates.idx_components_bad)
    else:
        print("No components were rejected.")

### Extract DF/F values

In [ ]:
#%% Extract DF/F values
#FIXME: "Oops!" printed when cnm2 not in locals (i.e. no refitting was done). Possibly this function never returns.
if "cnm2" in locals():
    cnm2.estimates.detrend_df_f(quantileMin=8, frames_window=250)
else:
    cnm.estimates.detrend_df_f(quantileMin=8, frames_window=250)

### Select only high quality components

In [ ]:
if "cnm2" in locals():
    cnm2.estimates.select_components(use_object=True)
else:
    cnm.estimates.select_components(use_object=True)

## Display final results

In [ ]:
if "cnm2" in locals():
    cnm2.estimates.nb_view_components(img=Cn, denoised_color='red')
else:
    cnm.estimates.nb_view_components(img=Cn, denoised_color='red')
print('you may need to change the data rate to generate this one: use jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 before opening jupyter notebook')

## Saving, closing, and creating denoised version
### You can save an hdf5 file with all the fields of the cnmf object. Use load_CNMF() to open the results again

In [ ]:
save_results = True
save_path = os.path.splitext(export_hd5_fpath)[0] + "_results.hdf5"
if save_results:
    if "cnm2" in locals():
        cnm2.save(save_path)
    else:
        cnm.save(save_path)
    print(f"saved to\n{save_path}")

### Stop cluster and clean up log files

In [ ]:
#%% STOP CLUSTER and clean up log files
cm.stop_server(dview=dview)
log_files = glob.glob('*_LOG_*')
for log_file in log_files:
    os.remove(log_file)

### Export parameters and metadata as json

In [ ]:
import json

In [ ]:
json_dict = opts_dict.copy()
json_dict["original_fnames"] = nd2_fpath
json_dict["rnr_win"] = win
json_dict["amplitude_threshold"] = amplitude_threshold

In [ ]:
json_export_dir, json_fname = os.path.split(export_hd5_fpath)
json_fname = ".".join(json_fname.split(".")[:-1]) + "_pars.json"
json_fpath = os.path.join(json_export_dir, json_fname)
print(f"Parameters will be saved under\n{json_fpath}")

In [ ]:
with open(json_fpath, 'w') as f:
    json.dump(json_dict, f, indent=4)
print(f"Saved parameters to\n{json_fpath}")

### View movie with the results
We can inspect the denoised results by reconstructing the movie and playing alongside the original data and the resulting (amplified) residual movie

In [ ]:
if "cnm2" in locals():
    cnm2.estimates.play_movie(images, q_max=99.9, gain_res=2,
                                      magnification=2,
                                      bpx=border_to_0,
                                      include_bck=False)
else:
    cnm.estimates.play_movie(images, q_max=99.9, gain_res=2,
                                  magnification=2,
                                  bpx=border_to_0,
                                  include_bck=False)

The denoised movie can also be explicitly constructed using:

In [ ]:
#%% reconstruct denoised movie
if "cnm2" in locals():
    denoised = cm.movie(cnm2.estimates.A.dot(cnm2.estimates.C) + \
                        cnm2.estimates.b.dot(cnm2.estimates.f)).reshape(dims + (-1,), order='F').transpose([2, 0, 1])
else:
    denoised = cm.movie(cnm.estimates.A.dot(cnm.estimates.C) + \
                        cnm.estimates.b.dot(cnm.estimates.f)).reshape(dims + (-1,), order='F').transpose([2, 0, 1])